# Overview

In a separate file we generate a dataframe that can be used for comparing segment vs kinesis.

This requires the output of that process, and provides a simpler area for investigating it (and hopefully less likely to crash).

# Prerequisites

* This uses ipython widgets that might not show up if you don't have the right jupyterlab extension installed


In [14]:
!pip install pyarrow

# Imports

In [8]:
# Safe imports
from datetime import datetime, timedelta, date

In [9]:
import pandas as pd
import numpy as np
import math

In [10]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [11]:
# Imports on files that might have dependencies that need installing
import data_pier_querying
from athena_querying import AthenaQuery
from athena_common_queries import *
import user_agents # this converts user agent from browser to mobile / desktop etc.

ModuleNotFoundError: No module named 'user_agents'

# Settings

In [12]:
load_merged_results_file_path = "merged_df_with_meta_20200209_0000_to_20200216_0000.gzip"

In [13]:
merged_df_with_meta = pd.read_parquet(load_merged_results_file_path)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support

In [7]:
merged_df_with_meta.head(2)

NameError: name 'merged_df_with_meta' is not defined

In [8]:
len(merged_df_with_meta)

NameError: name 'merged_df_with_meta' is not defined

In [9]:
merged_df_with_meta.dtypes

anonymous_id         object
event_name           object
page_url             object
date               category
s_count               int64
k_count               int64
page_type          category
slug               category
slug_root            object
ab_test            category
country_code       category
user_agent           object
device_family      category
os_family          category
os_version         category
browser_family     category
browser_version    category
is_bot              float16
dtype: object

In [10]:

def highlight_cols(cell):
    #use hex colours, or named ones to ensure excel compatibility on export
    if cell=="":
        return ""
    ci = min(100, int(abs(cell*10)))
    if abs(cell)<=2:
        return "color:green;"
    if cell <0:

        return "background-color:#%02x%02x%02x;" % (255,255-ci,255-ci)
    if cell>0:
        #return "background-color:cyan;"
        return "background-color:#%02x%02x%02x;" % (255-ci, 255-ci, 255)

    return "background-color:red;"

def colour_grouped_table(df):

    return df.style.applymap(highlight_cols, subset=["k_vs_s_%"])

In [11]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

# Search Feature

In [12]:
country_codes = merged_df_with_meta.country_code.unique().to_list()
country_codes

['sg', 'hk', '??', 'tw', 'ph', 'id']

In [13]:
top_level_slugs = list(merged_df_with_meta[merged_df_with_meta.page_type!="blog"].slug_root.unique())
top_level_slugs.sort()
event_types = list(merged_df_with_meta.event_name.unique())
event_types.sort()

page_types = list(merged_df_with_meta.page_type.unique())

num_events = len(merged_df_with_meta)

In [14]:

search_results_df = pd.DataFrame()

def on_search_button_click(b):
    #pandas likes lists, not tuples (at least for group by)
    print("searching")
    print("erm...")
    df = merged_df_with_meta
    
    anonymous_id = anonymous_user_input.value.strip()
    search_anonymous_id = bool(anonymous_id)
    
    event_types = list(event_type_select.value)
    search_event_types = len(event_types)>0
    
    slug_search_string = slug_search_input.value.strip()
    search_slug_by_string = bool(slug_search_string)
    
    country_codes = list(country_code_dropdown.value)
    search_country_codes =  len(country_codes)!=0
        
    top_level_slugs = list(top_level_slug_select.value)
    search_by_top_level_slugs = len(top_level_slugs)>0
    
    group_by_cols = list(group_by_select.value)
    do_group_by = len(group_by_cols)>0
    
    page_types = list(page_type_select.value)
    search_by_page_type = len(page_types)>0
    
    at_least_events = int(event_count_slider.value)
    
    print("%s %s"%(search_by_page_type, page_types))

    print("Events search")
    d = df[(( not search_anonymous_id) | (df.anonymous_id==anonymous_id)) \
           & ( (not search_event_types) | (df.event_name.isin(event_types))) \
          & (( not search_slug_by_string) | (df.slug.str.contains(slug_search_string))) \
            & (( not search_country_codes) | (df.country_code.isin(country_codes))) \
           & (( not search_by_top_level_slugs) | (df.slug_root.isin(top_level_slugs))) \
            & (( not search_by_page_type) | (df.page_type.isin(page_types))) \
           
           ]



    if do_group_by:
        #d = d.groupby(group_by_cols).sum()
        d = group_by_and_show_count_difference(d, group_by_cols)
        d = d[((d.s_count > at_least_events) | (d.k_count>at_least_events))]
        colour_grouped_table(d)
    search_results_df = d
    display(search_results_df)
    
    
    #return search_results_df
    print("done searching")
    
def on_reset_button_click(b):
    print("I would be resetting")

In [15]:
def button_click_placeholder(b):
    print("just chilling")

anonymous_user_input = widgets.Text(description = "Anonymous_id")
country_code_dropdown = widgets.SelectMultiple(
    options= country_codes,
    value=["sg","hk"],
    # rows=10,
    description='Country',
    disabled=False
)
search_button = widgets.Button(description='Search', on_click=on_search_button_click)
#reset_button = widgets.Button(description='Reset', on_click=on_reset_button_click)


search_button.on_click(on_search_button_click)

top_level_slug_select = widgets.SelectMultiple(options = top_level_slugs, description="slug")

slug_search_input = widgets.Text(description = "Slug like")

event_type_select = widgets.SelectMultiple(options=event_types, description="Event")

page_type_select = widgets.SelectMultiple(options=page_types, description="Page Type")


#event_count_box = widgets.Text(description="Min num events", )
#event_count_slider = widgets.FloatLogSlider(value=num_events, min=0, max=num_events, step=1, description="Num events", continuous_update=True, readout=True)
event_count_slider = widgets.FloatLogSlider(value=0, min=0, max=math.log10(num_events), description="At least num events")

#search_modes = ["Summary", "Summary Deduped", "Events",] # summary - > grouped with difference, events -> grouped by anon_id etc, 
#search_mode_dropdown = widgets.Dropdown(description="Search Mode", options=search_modes, value=search_modes[0])

search_options = widgets.Box([anonymous_user_input, country_code_dropdown ,
                              top_level_slug_select, slug_search_input,  event_type_select,
                             page_type_select,
                              event_count_slider
                             ])

search_options.layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap') #auto wrap

group_by_select = widgets.SelectMultiple(description="Group By", options = merged_df_with_meta.columns.to_list())

search_bar = widgets.VBox([search_options, group_by_select, widgets.Box([search_button])])
display(search_bar)

